In [4]:
!nvidia-smi

Mon Feb 28 15:55:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from google.colab import auth
auth.authenticate_user()

In [6]:
!gcloud config set project alecsharpie
!gcloud config list

Updated property [core/project].
[component_manager]
disable_update_check = True
[compute]
gce_metadata_read_timeout_sec = 0
[core]
account = alecsharpie@gmail.com
project = alecsharpie

Your active configuration is: [default]


In [7]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  70472      0 --:--:-- --:--:-- --:--:-- 70472
OK
69 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 69 not upgraded.
Need to get 12.0 MB of archives.
After this operation, 28.6 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 155320 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.40.0_amd64.deb ...
Unpacking gcsfuse (0.40.0) ...
Setting up gcsfuse (0.40.0) ...


In [8]:
!mkdir gcs_bucket
!gcsfuse --implicit-dirs image-datasets-alecsharpie gcs_bucket

2022/02/28 15:55:36.376461 Start gcsfuse/0.40.0 (Go version go1.17.6) for app "" using mount point: /content/gcs_bucket
2022/02/28 15:55:36.390734 Opening GCS connection...
2022/02/28 15:55:36.858356 Mounting file system "image-datasets-alecsharpie"...
2022/02/28 15:55:36.893614 File system has been successfully mounted.


In [9]:
!ls gcs_bucket

cassava_farmer	packages


In [ ]:
#!pip install git+https://github.com/alecsharpie/cassava_farmer.git --no-cache


In [10]:
!mkdir history
!mkdir models

In [14]:
from tensorflow.keras.utils import image_dataset_from_directory

def get_image_generator_local(
    batch_size,
    train_path='raw_data/cassava-leaf-disease-classification/train_images_mid'
):


    train_ds = image_dataset_from_directory(
        train_path,
        batch_size=batch_size,
        subset='training',
        validation_split=.20,
        seed=42,
        image_size=(512, 512),
    )

    class_names = train_ds.class_names

    train_size = train_ds.cardinality().numpy()
    train_ds = train_ds.unbatch().batch(batch_size)
    train_ds = train_ds.repeat()

    val_ds = image_dataset_from_directory(train_path,
                                          batch_size=32,
                                          subset='validation',
                                          validation_split=.20,
                                          seed=42,
                                          image_size=(512, 512))

    val_size = val_ds.cardinality().numpy()
    val_ds = val_ds.unbatch().batch(batch_size)
    val_ds = val_ds.repeat()
    return train_ds, train_size, val_ds, val_size

batch_size = 1

train_ds, train_size, val_ds, val_size = get_image_generator_local(batch_size, 'gcs_bucket/cassava_farmer/train_images')

steps_per_epoch = train_size // batch_size
validation_steps = val_size // batch_size




Found 21407 files belonging to 5 classes.
Using 17126 files for training.
Found 21407 files belonging to 5 classes.
Using 4281 files for validation.


In [17]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras import layers
from tensorflow.keras import Sequential


#from tensorflow.config import run_functions_eagerly

#from google.cloud import storage


def build_aug_eff_model(input_shape, output_classes):

    augmentation = Sequential([
        layers.RandomContrast(0.2),
        layers.RandomRotation(40),
        layers.RandomTranslation(0, 0.2),
        layers.RandomTranslation(0.2, 0),
        layers.RandomZoom(0.2, 0.2),
        layers.RandomFlip(mode="horizontal")
    ])

    dummy_input = layers.Input(shape=input_shape)

    topless_efficient_net = EfficientNetB0(include_top=False,
                                        weights='imagenet',
                                        input_tensor=dummy_input,
                                        pooling='max')

    aug_eff_model = Sequential([
        layers.Resizing(512, 512),
        augmentation,
        topless_efficient_net,
        layers.Dropout(0.2),
        layers.Dense(output_classes, activation='softmax')
    ])

    #top3_acc = functools.partial(top_k_categorical_accuracy, k=3)
    #top3_acc.__name__ = 'top3_acc'

    aug_eff_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])
                        #run_eagerly=True)

    aug_eff_model.build((None, 512, 512, 3))
    aug_eff_model.summary()

    aug_eff_model.layers[2].trainable = False

    return aug_eff_model

In [30]:
#from cassava_farmer.data import get_image_generator_local
#from cassava_farmer.model import build_aug_eff_model, save_model_to_gcp
#from cassava_farmer.gcs import storage_upload_file, storage_upload_folder

from tensorflow.keras.callbacks import EarlyStopping
import numpy as np


es = EarlyStopping(patience=10)

model = build_aug_eff_model((512, 512, 3), 5)

train_ds, train_size, val_ds, val_size = get_image_generator_local(8, 'gcs_bucket/cassava_farmer/train_images')
batch_size = 8

steps_per_epoch = train_size // batch_size
validation_steps = val_size // batch_size

# steps_per_epoch = 10
# validation_steps = 3

print('steps_per_epoch: ', steps_per_epoch)
print('validation_steps: ', validation_steps)

count_map = {
    '0': 1087,
    '1': 2189,
    '2': 2386,
    '3': 13158,
    '4': 2577
}

# balance dataset
avg_count = np.array(list(count_map.values())).mean()
class_weights = {k: (1 / v) * avg_count for k, v in count_map.items()}

history = model.fit(
  train_ds,
  epochs=100,
  #batch_size=batch_size,
  class_weight = class_weights,
  steps_per_epoch=steps_per_epoch,
  validation_data=val_ds,
  validation_steps=validation_steps,
  validation_batch_size=batch_size,
  callbacks=[es]).history


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_6 (Resizing)       (None, 512, 512, 3)       0         
                                                                 
 sequential_12 (Sequential)  (None, 512, 512, 3)       0         
                                                                 
 efficientnetb0 (Functional)  (None, 1280)             4049571   
                                                                 
 dropout_6 (Dropout)         (None, 1280)              0         
                                                                 
 dense_6 (Dense)             (None, 5)                 6405      
                                                                 
Total params: 4,055,976
Trainable params: 4,013,953
Non-trainable params: 42,023
_________________________________________________________________
Found 21407 files belonging to 5 class

ValueError: ignored

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history['loss'], label = 'train')
plt.plot(history['val_loss'], label = 'val')
plt.legend();

In [ ]:
import json
from datetime import datetime

history_file_name = f'history/{datetime.now().strftime("history_%Y-%m-%d_%H-%M-%S")}.json'
out_file = open(history_file_name, "w")
json.dump(history, out_file, indent="")
out_file.close()
#storage_upload_file(history_file_name)


print(history)
print('min accuracy', min(history['accuracy']))

model.save('models/aug_eff_model_test')

#storage_upload_folder('models/aug_eff_model_test')

In [ ]:
# from cassava_farmer.trainer import Trainer
# trainer = Trainer('colab')
# trainer.train(data_path = 'gcs_bucket/cassava_farmer/train_images')

In [ ]:
!zip -r /content/models/aug_eff_model_test_colab3.zip /content/models/aug_eff_model_test

from google.colab import files
files.download("/content/models/aug_eff_model_test_colab3.zip")

In [ ]:
# load previous history
import json

history = json.load(open('/content/history/history_2022-02-27_16-43-50'))

history

In [7]:
# home made model

from tensorflow.keras import layers
from tensorflow.keras import Sequential


def build_model():

  augmentation = Sequential([
          layers.RandomContrast(0.2),
          layers.RandomRotation(40),
          layers.RandomTranslation(0, 0.2),
          layers.RandomTranslation(0.2, 0),
          layers.RandomZoom(0.2, 0.2),
          layers.RandomFlip(mode="horizontal")
      ])


  model = Sequential([
                      layers.Resizing(512, 512),
                      augmentation,
                      layers.Conv2D(16, (5, 5), activation = 'relu'),
                      layers.MaxPooling2D(4),
                      layers.Conv2D(32, (5, 5), activation = 'relu'),
                      layers.MaxPooling2D(4),
                      layers.Conv2D(64, (5, 5), activation = 'relu'),
                      layers.MaxPooling2D(4),
                      layers.Flatten(),
                      layers.Dense(10, activation = 'relu'),
                      layers.Dropout(0.4),
                      layers.Dense(5, activation = 'softmax')
  ])

  model.compile(optimizer = 'adam',
                loss = 'sparse_categorical_crossentropy',
                metrics = 'accuracy')
  return model


Found 21407 files belonging to 5 classes.
Using 17126 files for training.
Found 21407 files belonging to 5 classes.
Using 4281 files for validation.


In [15]:
import numpy as np

label_map = {
    '0': 'cassava_bacterial_blight', #1087
    '1': 'cassava_brown_streak_disease', #2189
    '2': 'cassava_green_mottle', #2386
    '3': 'cassava_mosaic_disease', #13158
    '4': 'healthy' #2577
}

count_map = {
    '0': 1087,
    '1': 2189,
    '2': 2386,
    '3': 13158,
    '4': 2577
}

# balance dataset
avg_count = np.array(list(count_map.values())).mean()
class_weights = {k: (1 / v) * avg_count for k, v in count_map.items()}

In [8]:
test_model = build_model()
test_model.build((None, 512, 512, 3))
test_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_3 (Resizing)       (None, 512, 512, 3)       0         
                                                                 
 sequential_7 (Sequential)   (None, 512, 512, 3)       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 508, 508, 16)      1216      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 127, 127, 16)     0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 123, 123, 32)      12832     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 30, 30, 32)       0         
 g2D)                                                 

In [41]:
test_model = build_model()

# history = test_model.fit(
#                 train_ds,
#                 epochs=1,
#                 class_weights = class_weights
#                 #batch_size=batch_size,
#                 steps_per_epoch=steps_per_epoch,
#                 validation_data=val_ds,
#                 validation_steps=validation_steps,
#                 validation_batch_size=batch_size).history
#                 #callbacks=[es]

Epoch 1/3
17126/17126 [==============================] - 1396s 81ms/step - loss: 1.2122 - accuracy: 0.6177 - val_loss: 1.2476 - val_accuracy: 0.5896
Epoch 2/3
17126/17126 [==============================] - 1401s 82ms/step - loss: 1.2273 - accuracy: 0.6179 - val_loss: 1.2384 - val_accuracy: 0.5896
Epoch 3/3
17126/17126 [==============================] - 521s 30ms/step - loss: 1.1787 - accuracy: 0.6179 - val_loss: 1.1889 - val_accuracy: 0.6194
